In [1]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
DATA_DIR = 'C://Users//Yamil//Proyectos//Proyectos en Git//Memoria Ambulancias//Old Files//data//'
with open(DATA_DIR + 'dispatch_data_workable.pickle', 'rb') as f:
    df = pickle.load(f)
df = df[df['year'] == 2019]
HSdf = df[(df['FINAL_SEVERITY_LEVEL_CODE'] <= 3) & (df['INCIDENT_DISPOSITION_CODE'] != 87) & (df['INCIDENT_DISPOSITION_CODE'] != 90)]
LSdf = df[(df['FINAL_SEVERITY_LEVEL_CODE'] >= 4) & (df['INCIDENT_DISPOSITION_CODE'] != 87) & (df['INCIDENT_DISPOSITION_CODE'] != 90)]
len(HSdf), len(LSdf)

(523661, 827883)

In [4]:
def generateDemandShapefiles(original_df, demandPoints, zip_codes, save_dir):
    final_events_df = pd.DataFrame([])

    for code in zip_codes:
        zip_points = demandPoints[demandPoints['ZIPCODE'] == str(code)]
        sub_df = original_df[original_df['ZIPCODE'] == int(code)]
        
        new_df = sub_df[['INCIDENT_DATETIME', 'FINAL_SEVERITY_LEVEL_CODE']]
        samplepoints = zip_points.sample(len(new_df), replace=True)[['geometry', 'id']]
        new_df['geometry'] = list(samplepoints['geometry'])
        new_df['pointid'] = list(samplepoints['id'].astype('int64'))
        
        
        final_events_df = final_events_df.append(new_df, ignore_index = True)
        
    final_events_df = final_events_df.sort_values(by='INCIDENT_DATETIME')
    final_events_df.index = range(len(final_events_df))
    
    final_events_df['INCIDENT_DATETIME'] = final_events_df['INCIDENT_DATETIME'].dt.strftime('%m/%d/%Y %H:%M:%S')
    final_events_gdf = gpd.GeoDataFrame(final_events_df, geometry = 'geometry')
    final_events_gdf.crs = 'epsg:4326'
    final_events_gdf.to_file(save_dir)

In [10]:
DATA_DIR = 'C://Users//Yamil//Proyectos//Proyectos en Git//Memoria Ambulancias//Old Files//Generated Shapefiles//GeoTools//'
UniformDemandPoints = gpd.read_file(DATA_DIR + 'Uniform600m//Uniform600mDemandNew.geojson')
zip_codes = UniformDemandPoints['ZIPCODE'].unique()

In [11]:
DATA_DIR = 'C://Users//Yamil//Proyectos//Proyectos en Git//Memoria Ambulancias//Old Files//Generated Shapefiles//Dispatch Demand Shapefiles//final//'
names = ['HS', 'LS']
for i, original_df in enumerate([HSdf, LSdf]):
    generateDemandShapefiles(original_df, UniformDemandPoints, zip_codes, DATA_DIR + names[i] +"demands.shp")